# 🚀 HUFS-OOS-cs-student-ragbot: EC2 + Docker + GitHub Actions 배포 가이드

이 문서는 **HUFS-OOS-cs-student-ragbot** 프로젝트를 AWS EC2에 **Docker**로 패키징하고, **GitHub Actions(CI/CD)** 를 통해 자동 배포하며, **DuckDNS**로 도메인 연결까지 완료하는 전체 과정을 정리한 가이드입니다.

---

## 📦 프로젝트 요약

- **서버 배포 대상:** AWS EC2 Ubuntu 22.04 LTS
- **도메인 연결:** DuckDNS (`http://hufscomchatbot.duckdns.org`)
- **퍼블릭 IP:** `13.209.135.41`
- **SSH Key 파일:** `hufschatbot.pem`
- **백엔드 서버 포트:** `8000`

---

## ✅ 1. EC2 인스턴스 생성 및 접속

1. **AWS EC2** 대시보드 → Ubuntu 22.04 LTS 인스턴스 생성
2. **보안 그룹** 설정:
   - SSH: 포트 22
   - HTTP: 포트 80
   - FastAPI: 포트 8000
3. **탄력적 IP** 할당 후 연결 (예: `13.209.135.41`)
4. `.pem` 키파일 다운로드 (예: `hufschatbot.pem`)

### 🔐 SSH 접속

```bash
chmod 400 hufschatbot.pem
ssh -i "hufschatbot.pem" ubuntu@13.209.135.41